In [17]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

# Preprocessing of data

Here I will aim to form an array of arrays of tuples with each word and the following punctuation mark.

Example of single sentence:
[('I', ' '),
('love', ' '),
('kittens', ','),
('and', ' ').
('you', '?')]

In [18]:
from os import listdir
from os.path import isfile, join

datapath = 'target-sentences-tokenized'
onlyfiles = [join(datapath, f) for f in listdir(datapath) if isfile(join(datapath, f))]

In [19]:
data = []
punctuations = {',', '.', '!', '?', ':', ';'}
ignore = {'"', "'", '-', '–', '—', '(', ')'}
for file in onlyfiles:
    with open(file, 'r') as f:
        lines = f.readlines()
    for l in lines:
        words = l.split()
        curr_sent = []
        i = 0
        while i < len(words) - 1:
            current_pair = [words[i].lower()]
            if words[i].isnumeric():
                current_pair = ['NUM']
            if words[i] in ignore:
                i += 1
                continue
            if words[i + 1] in punctuations:
                current_pair.append(words[i + 1])
                i += 1
            elif words[i + 1] in ignore:
                current_pair.append(' ')
                i += 1
            else:
                current_pair.append(' ')
            curr_sent.append(tuple(current_pair))
            i += 1
        data.append(curr_sent)

In [20]:
data[:10]

[[('пишаюся', ','), ('бути', ' '), ('греко', ' '), ('католиком', '!')],
 [('греко', ' '),
  ('католики', ' '),
  ('вміють', ' '),
  ('обирати', ' '),
  ('дуже', ' '),
  ('розумних', ' '),
  ('предстоятелів', ':'),
  ('попереднім', ' '),
  ('був', ' '),
  ('блаженніший', ' '),
  ('любомир', ' '),
  ('гузар', ' '),
  (',', ' '),
  ('а', ' '),
  ('зараз', ' '),
  ('блаженніший', ' '),
  ('святослав', ' '),
  ('шевчук', ' ')],
 [('хочу', ' '),
  ('поділитися', ' '),
  ('із', ' '),
  ('вами', ' '),
  ('цитатою', ' '),
  ('блаженнішого', ' '),
  ('святослава', ' '),
  ('ще', ' '),
  ('перед', ' '),
  ('першим', ' '),
  ('туром', ':')],
 [('«', ' '),
  ('прохаю', ':'),
  ('ніколи', ' '),
  ('не', ' '),
  ('голосуйте', ' '),
  ('проти', ' ')],
 [('бо', ' '),
  ('голосуючи', ' '),
  ('проти', ' '),
  (',', ' '),
  ('ви', ','),
  ('де-факто', ','),
  ('проголосуєте', ' '),
  ('проти', ' '),
  ('свого', ' '),
  ('майбутнього', '.')],
 [('і', ' '),
  ('це', ' '),
  ('абсолютно', ' '),
  ('неконстр

In [21]:
len(data)

29760

---------------

In [22]:
# split data into training and validation set in the ratio 90:10
train_set, test_set = train_test_split(data, train_size=0.9, test_size=0.009, random_state=101)

# create list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup for sent in test_set for tup in sent]
print(len(train_tagged_words))
print(len(test_tagged_words))

317054
3268


In [23]:
# check some tagged words.
train_tagged_words[:5]

[('тобі', ' '), ('кажу', ','), ('страшно', '!'), ('моє', ' '), ('життя', ' ')]

In [24]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word, tag in train_tagged_words}
print(len(tags))
print(tags)

# check total words in vocabulary
vocab = {word for word, tag in train_tagged_words}

7
{'.', ':', ' ', '!', ',', '?', ';'}


In [25]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)


    return (count_w_given_tag, count_tag)


# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

print(tags_matrix)

[[2.9994618e-02 3.6698144e-03 8.4826541e-01 1.2134854e-02 9.9574305e-02
  6.0674269e-03 2.4465431e-04]
 [1.3048636e-02 2.9655991e-03 8.5943061e-01 1.1862396e-02 1.0616845e-01
  6.5243179e-03 0.0000000e+00]
 [7.5288720e-02 5.9332168e-03 7.8696495e-01 5.5724396e-03 1.1714672e-01
  7.3018176e-03 1.7921217e-03]
 [2.6521061e-02 3.1201248e-03 8.5751432e-01 1.2480499e-02 9.5163807e-02
  4.1601663e-03 1.0400416e-03]
 [1.4186402e-02 2.1760019e-03 8.9326286e-01 5.4258746e-03 8.1133783e-02
  3.0803143e-03 7.3475385e-04]
 [2.1626703e-02 2.8208746e-03 8.6694878e-01 7.5223320e-03 9.5439583e-02
  5.6417491e-03 0.0000000e+00]
 [8.1632650e-03 8.1632650e-03 8.9183676e-01 4.0816325e-03 8.5714288e-02
  2.0408162e-03 0.0000000e+00]]


In [26]:
# convert the matrix to a df for better readability
# the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns=list(tags), index=list(tags))
display(tags_df)

,.,:,,!,",",?,;
.,0.029995,0.003670,0.848265,0.012135,0.099574,0.006067,0.000245
:,0.013049,0.002966,0.859431,0.011862,0.106168,0.006524,0.000000
,0.075289,0.005933,0.786965,0.005572,0.117147,0.007302,0.001792
!,0.026521,0.003120,0.857514,0.012480,0.095164,0.004160,0.001040
",",0.014186,0.002176,0.893263,0.005426,0.081134,0.003080,0.000735
?,0.021627,0.002821,0.866949,0.007522,0.095440,0.005642,0.000000
;,0.008163,0.008163,0.891837,0.004082,0.085714,0.002041,0.000000


In [27]:
from tqdm import tqdm


def Viterbi(words, train_bag=train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    print(T)
    for key, word in enumerate(tqdm(words)):
        # initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag] + tags_df.loc['?', tag] + tags_df.loc['!', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            count_word_tag, count_tag = word_given_tag(words[key], tag)
            emission_p = count_word_tag / count_tag
#             emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))


In [28]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times

# choose random 10 numbers
rndom = [random.randint(1, len(test_set)) for x in range(10)]

# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

# Here We will only test 10 sentences to check the accuracy
# as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]

accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ', accuracy * 100)

['.', ':', ' ', '!', ',', '?', ';']


100%|██████████| 151/151 [00:45<00:00,  3.30it/s]

Time taken in seconds:  45.814030170440674
Viterbi Algorithm Accuracy:  78.80794701986756


In [29]:
tagged_seq

[('і', ' '),
 ('тут', ' '),
 ('до', ' '),
 ('кімнати', ' '),
 ('зайшов', ' '),
 ('якийсь', ' '),
 ('громадянин', ' '),
 ('так', ' '),
 ('що', ' '),
 ('ж', ' '),
 ('робити', ' '),
 ('а', ' '),
 ('з', ' '),
 ('приводу', ' '),
 ('ту', ' '),
 ('NUM', ' '),
 ('розбилися', ' '),
 ('гідні', ' '),
 ('колеги', ' '),
 ('геббельса', '.'),
 ('які', ' '),
 ('в', ' '),
 ('промислових', ' '),
 ('масштабах', ' '),
 ('готували', ' '),
 ('весь', ' '),
 ('той', ' '),
 ('потік', ' '),
 ('пропаганди', ' '),
 ('про', ' '),
 ('нашу', ' '),
 ('країну', ' '),
 ('включаючи', ' '),
 ('фейки', ' '),
 ('про', ' '),
 ('збитий', '.'),
 ('боїнг', '.'),
 ('ось', ' '),
 ('це', ' '),
 ('іронія', '.'),
 ('долі', ' '),
 ('цікавим', ' '),
 ('також', ' '),
 ('є', ' '),
 ('те', ','),
 ('що', ' '),
 ('NUM', ' '),
 ('%', ' '),
 ('висловили', '.'),
 ('загальну', ' '),
 ('недовіру', ','),
 ('церквам', '.'),
 ('проте', ' '),
 ('лише', ' '),
 ('NUM', ' '),
 ('%', ' '),
 ('визначились', '.'),
 ('із', ' '),
 ('тим', ' '),
 ('що', ' 

In [30]:
len(test_tagged_words)

151

In [31]:
def measure_sign(predicted_seq, test_seq, punctuation_sign):
    test_pos = set([i for i, (_, second) in enumerate(test_seq) if second == punctuation_sign])
    predicted_pos = set([i for i, (_, second) in enumerate(predicted_seq) if second == punctuation_sign])
    
    # how many times was a punctuation sign restored correctly
    true_positives = len(test_pos & predicted_pos)
    
    # how many times wasn't a punctuation sign restored when it should habe been
    false_negatives = len(test_pos - predicted_pos)
    
    # how many times was a punctuation sign restored when it shouldn't have been there
    false_positives = len(predicted_pos - test_pos)
    
    no_test_pos = set([i for i, (_, second) in enumerate(test_seq) if second != punctuation_sign])
    no_predicted_pos = set([i for i, (_, second) in enumerate(predicted_seq) if second != punctuation_sign])
    true_negatives = len(no_test_pos & no_predicted_pos)
    
    return true_positives, false_positives, false_negatives, true_negatives


def standard_tests(true_positives, false_positives, false_negatives, true_negatives):
    accuracy = (true_positives + true_negatives) / \
        (true_positives + true_negatives + false_positives + false_negatives) \
        if (true_positives + true_negatives + false_positives + false_negatives) > 0 else np.nan
    precision = true_positives / (true_positives + false_positives) \
        if (true_positives + false_positives) > 0 else np.nan
    recall = true_positives / (true_positives + false_negatives) \
        if (true_positives + false_negatives) > 0 else np.nan
    f_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else np.nan
    
    return f"{accuracy:.3f}", f"{precision:.3f}", f"{recall:.3f}", f"{f_score:.3f}"

In [32]:
def evaluate(predicted_seq, test_seq):
    tp, fp, fn, tn = 0,0,0,0
    print("{:<16} {:<9} {:<12} {:<9} {:<9}".format("Punctuation", "Accuracy", "Precision", "Recall", "F-score"))
    for tag in tags:
        true_positives, false_positives, false_negatives, true_negatives = \
            measure_sign(predicted_seq, test_seq, tag)
        tp += true_positives
        fp += false_positives
        fn += false_negatives
        tn += true_negatives
        accuracy, precision, recall, f_score = \
            standard_tests(true_positives, false_positives, false_negatives, true_negatives)
        print("{:<16} {:<9} {:<12} {:<9} {:<9}".format(tag, accuracy, precision, recall, f_score))

    accuracy, precision, recall, f_score = standard_tests(tp, fp, fn, tn)
    print("{:<16} {:<9} {:<12} {:<9} {:<9}".format("Overall", accuracy, precision, recall, f_score))

evaluate(tagged_seq, test_run_base)

Punctuation      Accuracy  Precision    Recall    F-score  
.                0.874     0.067        0.167     0.095    
:                1.000     nan          nan       nan      
                 0.828     0.906        0.891     0.898    
!                0.993     0.000        nan       nan      
,                0.887     0.375        0.200     0.261    
?                0.993     nan          0.000     nan      
;                1.000     nan          nan       nan      
Overall          0.939     0.788        0.788     0.788    


In [33]:
# Code to test all the test sentences
# (takes alot of time to run s0 we won't run it here)
# tagging the test sentences()
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]


print('Starting testing')
# start = time.time()
tagged_seq = Viterbi(test_untagged_words)
# end = time.time()
# difference = end-start

# print("Time taken in seconds: ", difference)


Starting testing
['.', ':', ' ', '!', ',', '?', ';']


100%|██████████| 3268/3268 [16:31<00:00,  3.29it/s]


In [34]:
tagged_seq

[('презентація', ' '),
 ('може', ' '),
 ('бути', ' '),
 ('взагалі', ' '),
 ('без', ' '),
 ('картинок', ' '),
 ('і', ' '),
 ('виглядати', ' '),
 ('круто', ' '),
 ('якщо', ' '),
 ('доносить', ' '),
 ('ідею', ' '),
 ('її', ' '),
 ('вихід', ' '),
 ('супроводжувався', ' '),
 ('невеликим', ' '),
 ('інтерв’ю', ' '),
 ('яке', ' '),
 ('взяв', ' '),
 ('у', ' '),
 ('таяни', '.'),
 ('алан', ' '),
 ('бадоєв', '.'),
 ('відкопайте', '.'),
 ('свій', ' '),
 ('стержень', '.'),
 ('свої', ' '),
 ('цінності', ' '),
 ('принципи', ' '),
 ('ідеологію', '.'),
 ('і', ' '),
 ('не', ' '),
 ('зраджуйте', '.'),
 ('його', ' '),
 ('це', ' '),
 ('надзвичайно', ' '),
 ('важливо', ' '),
 ('і', ' '),
 ('це', ' '),
 ('те', ','),
 ('що', ' '),
 ('тримає', ' '),
 ('тебе', ' '),
 ('на', ' '),
 ('плаву', '.'),
 ('під', ' '),
 ('час', ' '),
 ('змін', ' '),
 ('в', ' '),
 ('той', ' '),
 ('час', ' '),
 ('коли', ' '),
 ('всі', ' '),
 ('ідуть', ' '),
 ('на', ' '),
 ('дно', ','),
 ('його', ' '),
 ('пояс', ' '),
 ('відсвічував', '.')

In [35]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_tagged_words) if i == j]

accuracy = len(check) / len(tagged_seq)
print('Viterbi Algorithm Accuracy: ', accuracy * 100)

Viterbi Algorithm Accuracy:  74.66340269277846


In [36]:
evaluate(tagged_seq, test_tagged_words)

Punctuation      Accuracy  Precision    Recall    F-score  
.                0.856     0.152        0.284     0.198    
:                0.996     0.200        0.091     0.125    
                 0.780     0.856        0.874     0.865    
!                0.993     0.500        0.136     0.214    
,                0.876     0.440        0.212     0.286    
?                0.992     0.000        0.000     nan      
;                0.999     0.000        0.000     nan      
Overall          0.928     0.747        0.747     0.747    


In [37]:
comma_before = {'щоб', 'що', 'але', 'а', 'яке', 'який', 'яка', 'які', 'котра', 'котрий', 'котре', 'котрі', 'або', 'бо', 'то', 'ласка'}
comma_before_and_after = {'можливо', 'може', 'здається'}

def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(tqdm(words)):
        # initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag] + tags_df.loc['!', tag] + tags_df.loc['?', tag] + tags_df.loc[';', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            count_word_tag, count_tag = word_given_tag(word, tag)
            emission_p = count_word_tag / count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = T[p.index(pmax)]
        
        if state_max == ' ': # if we predicted space, use rule-based prediction based on the general rules of Ukrainian language
            if words[key + 1] in comma_before or words[key + 1] in comma_before_and_after:
                state_max = ','
            elif word in comma_before_and_after:
                state_max = ','
         
        state.append(state_max)
    return list(zip(words, state))

In [38]:
tagged_seq = Viterbi_rule_based(test_untagged_words)

100%|██████████| 3268/3268 [16:31<00:00,  3.30it/s]


In [39]:
check = [i for i, j in zip(tagged_seq, test_tagged_words) if i == j]

accuracy = len(check) / len(tagged_seq)
print('Viterbi Algorithm Accuracy: ', accuracy * 100)

Viterbi Algorithm Accuracy:  75.9485924112607


In [40]:
evaluate(tagged_seq, test_tagged_words)

Punctuation      Accuracy  Precision    Recall    F-score  
.                0.856     0.152        0.284     0.198    
:                0.996     0.200        0.091     0.125    
                 0.795     0.878        0.865     0.872    
!                0.993     0.500        0.136     0.214    
,                0.887     0.523        0.380     0.440    
?                0.992     0.000        0.000     nan      
;                0.999     0.000        0.000     nan      
Overall          0.931     0.759        0.759     0.759    
